# Aplicaciones usando Dash

Un buen repositorio para explorar ejemplos y prácticas es: https://github.com/Coding-with-Adam/Dash-by-Plotly , ya que también explica su contenido en el canal de Youtube **Charming Data**. 

Las aplicaciones se basan en **Components** que son mostradas a través de un **Layout** y que interactúan entre sí por medio del **Callback**.




*   Components: Selectores de fecha, controles deslizantes, casilla de verificación, botones, dropdown.
*   Layout: Personaliza la forma en que muestran los componentes en la app.
*   Callback: Le da vida a la aplicación, haciéndola funcional e interactiva.




En resumen: primero se crean los components, los cuales se asignan al Layout para ser mostrados en la app usando callbacks (Input-Output).

Además, el callback se constituye por dos partes: 

1.   Decoradores (Input-Output) con el ID y la propiedad del component que se desea usar.
2.   Funciones con la firma o argumento del callback y el valor de retorno.



Es importante comprender que la función argumento siempre viene de la propiedad del component de Input y que el return value se asigna al Output, de esta manera es como funciona el callback. Entonces se necesitan múltiples argumentos si se trabaja con múltiples Inputs.

In [3]:
%%capture
!pip install dash
!pip install dash_bootstrap_components
!pip install jupyter-dash
!pip install pyngrok --quiet
!pip install geopandas
!pip install -U kaleido

In [2]:
import dash
from dash import Dash, dcc, Output, Input,html  
import dash_bootstrap_components as dbc    
import plotly.express as px
import pandas as pd
import geopandas as gpd
import plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from pyngrok import ngrok
import sys
import numpy as np
import os

In [8]:
"""Debido a que se usó un ambiente alojado en la nube (Colab-Research) para implementar la aplicación es necesario utilizar un auth-token  para
 generar el túnel y poder visualizar la app desde el ordenador personal mediante el puerto que se desee, para esto se utiliza la librería pyngrok"""

ngrok.kill()
NGROK_AUTH_TOKEN = "2AD0P49GHZ4ac4XeBmTq8WB9pRI_63F4wyuSX5WAoW9iBSxqf"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [5]:
%%writefile test_app.py


import dash
from dash import Dash, dcc, Output, Input, html  
import dash_bootstrap_components as dbc    
import plotly.express as px
import pandas as pd
import geopandas as gpd
import plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from pyngrok import ngrok
import sys
import numpy as np
import os
import plotly.io as pio

pd.set_option('max_rows',20)
pio.renderers.default = "browser"


app = Dash(__name__, external_stylesheets=[dbc.themes.MATERIA])
#app.title = 'Dashboard de Covid-19'


colors = {
    'background': '#f0ad4e',
    'bodyColor':'#343a40',
    'text': '#343a40'
}



#Lectura de datos

CONF_URL = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
DEAD_URL = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
RECV_URL = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'


covid_conf_ts = pd.read_csv(CONF_URL)
covid_dead_ts = pd.read_csv(DEAD_URL)
covid_recv_ts = pd.read_csv(RECV_URL)


#obtener datos en formato de series temporales limpias para el país
def process_data(data,cntry='Costa Rica',window=3):
    conf_ts = data
    conf_ts_cntry = conf_ts[conf_ts['Country/Region']==cntry]
    final_dataset = conf_ts_cntry.T[4:].sum(axis='columns').diff().rolling(window=window).mean()[40:]
    df = pd.DataFrame(final_dataset,columns=['Total'])
    return df

#obtener el total global de casos confirmados, recuperados y muertos
def get_overall_total(df):
    return df.iloc[:,-1].sum()

conf_overall_total = get_overall_total(covid_conf_ts)
dead_overall_total = get_overall_total(covid_dead_ts)
recv_overall_total = get_overall_total(covid_recv_ts)
print('Confirmados Globales:',conf_overall_total)
print('Fallecidos Globales:',dead_overall_total)
print('Recuperados Globables:',recv_overall_total)

#obtener el total de confirmados, recuperados y muertos para el país 
def get_cntry_total(df,cntry='Costa Rica'):
    return df[df['Country/Region']==cntry].iloc[:,-1].sum()

cntry = 'Costa Rica'
conf_cntry_total = get_cntry_total(covid_conf_ts,cntry)
dead_cntry_total = get_cntry_total(covid_dead_ts,cntry)
recv_cntry_total = get_cntry_total(covid_recv_ts,cntry)
print(f'{cntry} Confirmados:',conf_cntry_total)
print(f'{cntry} Fallecidos:',dead_cntry_total)
print(f'{cntry} Recuperados:',recv_cntry_total)



#Crear componentes como títulos, dropdown, slider, y cards para asignarlos al Layout.

titulo = dcc.Markdown(children='')
grafico = dcc.Graph(figure={})
desplegable = dcc.Dropdown(options=['Costa Rica', 'Guatemala'],
                        value='Costa Rica',  # initial value displayed when page first loads
                        clearable=False)

#dia = 31
dias = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31']#[str(x) for x in range(dia + 1)]                       
desp_dia = dcc.Dropdown(options=dias,
                        value='01',  # initial value displayed when page first loads
                        clearable=False)

#mes = 12
meses = ['01','02','03','04','05','06','07','08','09','10','11','12'] #[str(x) for x in range(mes + 1)]                       
desp_mes = dcc.Dropdown(options=meses,
                        value='05',  # initial value displayed when page first loads
                        clearable=False)

                      
desp_año = dcc.Dropdown(options=['2020', '2021','2022'],
                        value='2021',  # initial value displayed when page first loads
                        clearable=False)


def get_page_heading_style():
    return {'backgroundColor': colors['background']}


def get_page_heading_title():
    return html.H1(children='Dashboard de Covid-19',
                                        style={
                                        'textAlign': 'center',
                                        'color': colors['text']
                                    })

def get_page_heading_subtitle():
    return html.Div(children='Visualización de datos de Covid-19',
                                         style={
                                             'textAlign':'center',
                                             'color':colors['text']
                                         })

def generate_page_header():
    main_header =  dbc.Row(
                            [
                                dbc.Col(get_page_heading_title(),md=12)
                            ],
                            align="center",
                            style=get_page_heading_style()
                        )
    subtitle_header = dbc.Row(
                            [
                                dbc.Col(get_page_heading_subtitle(),md=12)
                            ],
                            align="center",
                            style=get_page_heading_style()
                        )
    header = (main_header,subtitle_header)
    return header


def get_country_list():
    return covid_conf_ts['Country/Region'].unique()

def create_dropdown_list(cntry_list):
    dropdown_list = []
    for cntry in sorted(cntry_list):
        tmp_dict = {'label':cntry,'value':cntry}
        dropdown_list.append(tmp_dict)
    return dropdown_list

def get_country_dropdown(id):
    return html.Div([
                        html.Label('Selecciona el país'),
                        dcc.Dropdown(id='my-id'+str(id),
                            options=create_dropdown_list(get_country_list()),
                            value='Costa Rica'
                        ),
                        html.Div(id='my-div'+str(id))
                    ])

def fig_world_trend(cntry='Costa Rica',window=3):
    df = process_data(data=covid_conf_ts,cntry=cntry,window=window)
    df.head(10)
    if window==1:
        yaxis_title = "Casos Diarios"
    else:
        yaxis_title = "Casos diarios ({}-día MA)".format(window)
    fig = px.line(df, y='Total', x=df.index, title='Tendencia de casos diarios confirmados para {}'.format(cntry),height=600,color_discrete_sequence =['navy']) #,color_discrete_sequence =['navy']
    fig.update_layout(title_x=0.5,paper_bgcolor='#fff',xaxis_title="Fecha",yaxis_title=yaxis_title) #,plot_bgcolor='#fff'
    return fig


def graph1():
    return dcc.Graph(id='graph1',figure=fig_world_trend('Costa Rica'))


def generate_card_content(card_header,card_value,overall_value):
    card_head_style = {'textAlign':'center','fontSize':'150%'}
    card_body_style = {'textAlign':'center','fontSize':'200%'}
    card_header = dbc.CardHeader(card_header,style=card_head_style)
    card_body = dbc.CardBody(
        [
            html.H5(f"{int(card_value):,}", className="card-title",style=card_body_style),
            html.P(
                "Global: {:,}".format(overall_value),
                className="card-text",style={'textAlign':'center'}
            ),
        ]
    )
    card = [card_header,card_body]
    return card


def generate_cards(cntry='Costa Rica'):
    conf_cntry_total = get_cntry_total(covid_conf_ts,cntry)
    dead_cntry_total = get_cntry_total(covid_dead_ts,cntry)
    recv_cntry_total = get_cntry_total(covid_recv_ts,cntry)
    cards = html.Div(
        [
            dbc.Row(
                [
                    dbc.Col(dbc.Card(generate_card_content("Confirmados",conf_cntry_total,conf_overall_total), color="warning", inverse=True),md=dict(size=2)),
                    dbc.Col(dbc.Card(generate_card_content("Fallecidos",dead_cntry_total,dead_overall_total),color="dark", inverse=True),md=dict(size=2)),
                ],
                className="mb-4",
            ),
        ],id='card1'
    )
    return cards



def get_slider():
    return html.Div([  
                        dcc.Slider(
                            id='my-slider',
                            min=1,
                            max=15,
                            step=None,
                            marks={
                                1: '1-día',
                                3: '3-días',
                                5: '5-días',
                                7: '1-semana',
                                14: '1-quincena'
                            },
                            value=3,
                        ),
                        html.Div([html.Label('Selecciona el promedio en la ventana')],id='my-div'+str(id),style={'textAlign':'center'})
                    ])


#Generar el diseño (Layout) utilizando los componentes anteriores

def generate_layout():
    page_header = generate_page_header()
    layout = dbc.Container(
        [
            page_header[0],
            page_header[1],
            html.Hr(),
            generate_cards(),
            html.Hr(),
            dbc.Row(
                [
                    dbc.Col(get_country_dropdown(id=1),md=dict(size=4,offset=4))                    
                ]
            
            ),
            dbc.Row(
                [                
                    
                    dbc.Col(graph1(),md=dict(size=6,offset=3))
        
                ],
                align="center",

            ),
            dbc.Row(
                [
                    dbc.Col(get_slider(),md=dict(size=4,offset=4))                    
                ]
            
            ),
            dbc.Row(
                [
                    dbc.Col([titulo], width=6)
                ], 
                justify='center'
             ),
            dbc.Row(
                [
                    dbc.Col([grafico], width=12)
                ]
             ),
             dbc.Row(
                    dbc.Col(html.Div("Selecciona el día - mes - año que desea graficar para Costa Rica o Guatemala"),width=12)
             ),
             dbc.Row(
                 [  
                    dbc.Col([desp_dia], width=1),
                    dbc.Col([desp_mes], width=1),
                    dbc.Col([desp_año], width=1),
                    dbc.Col([desplegable], width=2)
                 ], 
                 #justify='center'
              ),
              dbc.Row(
                    dbc.Col(html.Div("  "),width=12)
             ),

        ],fluid=True,style={'backgroundColor': 'white'}
    )
    return layout  


app.layout = generate_layout()

#Generar el callback para los componentes interactivos  

@app.callback(
    [Output(component_id='graph1',component_property='figure'), #line chart
    Output(component_id='card1',component_property='children'),#overall card numbers
    Output(grafico, 'figure'), #map
    Output(titulo, 'children')
    ],
    
    [Input(component_id='my-id1',component_property='value'), #dropdown
     Input(component_id='my-slider',component_property='value'),#slider
     Input(desplegable, 'value'), #dropdown
     Input(desp_dia,'value'),
     Input(desp_mes,'value'),
     Input(desp_año,'value')
     ]
)

#Generar la función de argumento para que la app se actualize con los cambios del usuario

def update_output_div(input_value1,input_value2, pais,dia,mes,año):
  if pais == 'Costa Rica':
    data_acums = pd.read_csv('mediaMovClean50.txt', decimal = '.', sep=',')
    df_admin = gpd.read_file('/content/provincias.json')
    fig = px.choropleth(data_frame=data_acums,
                        geojson=df_admin,
                        featureidkey='properties.name',
                        locations='departamento',
                        hover_name='departamento',
                        height=600,
                        color=str(dia+'/'+mes+'/'+año),
                        color_continuous_scale='Blues') # las funciones argumento vienen de la propiedad componente del Input

    
    
        #Se mueve la barra de colores
    fig.update_layout(coloraxis_colorbar_x=0.79, 
                    font=dict(size=20,
                    color="black")
                    )
    #Se aumenta la resolución
    fig.update_geos(resolution=110)

    #Se ajusta el margen de la imagen
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

        #Se eliminan los otros paises y limites que vienen por defecto en el choropleth para que sólo aparezca Guatemala
    fig.update_geos(showcountries=False, showcoastlines=False, showland=False, showframe = False, framewidth = 10, fitbounds="locations")
  
  elif pais =='Guatemala':
    data_acums = pd.read_csv('media_movGT.txt', decimal = '.', sep=',')
    df_admin = gpd.read_file('/content/stanford-vx042ws8701-geojson.json')

    fig = px.choropleth(data_frame=data_acums,
                        geojson=df_admin,
                        featureidkey='properties.name_1',
                        locations='departamento',
                        hover_name='departamento',
                        height=600,
                        color=str(dia+'/'+mes+'/'+año),
                        #animation_frame = ''
                        color_continuous_scale='Blues') # las funciones argumento vienen de la propiedad componente del Input

    
    
        #Se mueve la barra de colores
    fig.update_layout(coloraxis_colorbar_x=0.79, 
                    font=dict(size=20,
                    color="black")
                    )
    #Se aumenta la resolución
    fig.update_geos(resolution=110)

    #Se ajusta el margen de la imagen
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

        #Se eliminan los otros paises y limites que vienen por defecto en el choropleth para que sólo aparezca Guatemala
    fig.update_geos(showcountries=False, showcoastlines=False, showland=False, showframe = False, framewidth = 10, fitbounds="locations")

   
    #El return va a la propiedad componente del Output que en este caso van a ser las figuras, las terjetas y el nombre del país
  return fig_world_trend(input_value1,input_value2),generate_cards(input_value1), fig, '# '+pais
    





if __name__=='__main__':
    app.run_server(debug=True, port=8051)


Writing test_app.py


In [6]:
ngrok.connect(8051)

<NgrokTunnel: "http://f98e-34-125-211-213.ngrok.io" -> "http://localhost:8051">

In [7]:
!python test_app.py

Confirmados Globales: 577372223
Fallecidos Globales: 6400325
Recuperados Globables: 0
Costa Rica Confirmados: 1028375
Costa Rica Fallecidos: 8720
Costa Rica Recuperados: 0
Dash is running on http://127.0.0.1:8051/

 * Serving Flask app "test_app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
Confirmados Globales: 577372223
Fallecidos Globales: 6400325
Recuperados Globables: 0
Costa Rica Confirmados: 1028375
Costa Rica Fallecidos: 8720
Costa Rica Recuperados: 0
